# Regression Coefficients - Revisited

## Lesson Objectives

By the end of this lesson, students will be able to:
- Use scikit-learn v1.1's simplified toolkit.
- Extract and visualize coefficients from sklearn regression model. 
- Control panda's display options to facilitate interpretation.


## Introduction

- At the end of last stack, we dove deep into linear regression models and their assumptions. We introduced a new package called statsmodels, which produced a Linear Regression model using "Ordinary-Least-Squared (OLS)". 
- The model included a robust statistical summary that was incredibly informative as we critically diagnosed our regression model and if we met the assumptions of linear regression.
- This stack, we will be focusing on extracting insights from our models: both by examining parameters/aspects of the model itself, like the coefficients it calculated, but also by applying some additional tools and packages specifically designed to explain models. 

- Most of these tools are compatible with the scikit-learn ecosystem but are not yet available for statsmodels.

Since we are not focusing on regression diagnostics this week, we will shift back to using scikit-learn models. Scikit-learn recently released version 1.1.1, which added several helpful tools that will simplify our workflow. 

Let's review some of these key updates as we rebuild our housing regression model from week 16.


# Confirming Package Versions

- All packages have a version number that indicates which iteration of the package is currently being used.
    - If you import an entire package, you can use the special method `package.__version__` (replace package with the name of the package you want to check).
- The reason this is important is that as of the writing of this stack, Google Colab is still using a version of python that is too old to support the newest scikit-learn.
    - You can check which version of python you are using by running the following command in a jupyter notebook:
        - `!python --version`
        - Note: if you remove the `!`, you can run this command in your terminal.

- If you run the following code on Google Colab and on your local computer, you can compare the version numbers. 
        
<img src="colab_versions.png" width=400px>

- Now, run the following block of code in a jupyter notebook on your local machine to confirm that you have Python 3.8.13 and sklearn v1.1.1.


In [ ]:
# Run the following command on your local computer to 
import sklearn
print(f"sklearn version: {sklearn.__version__}")
!python --version


>- If you have a Python 3.7 or an earlier version of scikit-learn, please revisit the "`<Insert the name of the "week" of content on the LP for installation>`". 
    - See the "`Updating Your Dojo-Env Lesson` [Note to Brenda: does not exist yet - see 1:1 doc for question on handling multiple envs] for how to remove your current dojo-env and replace it with the new one.

# Extracting Coefficients from LinearRegression in scikit-learn

## Highlighted Changes  - scikit-learn v1.1

- The single biggest change in the updated sklearn is a fully-functional `.get_feature_names_out()` method in the `ColumnTransformer`.
    - This will make it MUCH easier for us to extract our transformed data as dataframes and to match up the feature names to our models' coefficients.
- There are some additional updates that are not pertinent to this stack, but if you are curious, you can find the [details on the new release here](https://scikit-learn.org/stable/auto_examples/release_highlights/plot_release_highlights_1_1_0.html).

## New and Improved `ColumnTransformer` 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

## Customization Options
plt.style.use(['fivethirtyeight','seaborn-talk'])
mpl.rcParams['figure.facecolor']='white'

## additional required imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import metrics

In [ ]:
## Load in the King's County housing dataset and display the head and info
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSEZQEzxja7Hmj5tr5nc52QqBvFQdCAGb52e1FRK1PDT2_TQrS6rY_TR9tjZjKaMbCy1m5217sVmI5q/pub?output=csv")

## Dropping some features for time
df = df.drop(columns=['date'])
display(df.head(),df.info())

In [ ]:
## Make the house ids the index
df = df.set_index('id')

In [ ]:
## Treating zipcode as a category
df['zipcode'] = df['zipcode'].astype(str)

### Train Test Split

In [ ]:
## Make x and y variables
y = df['price'].copy()
X = df.drop(columns=['price']).copy()

## train-test-split with random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=321)
X_train.head()

### Preprocessing + ColumnTransformer

In [ ]:
## make categorical selector and verifying it works 
cat_sel = make_column_selector(dtype_include='object')
cat_sel(X_train)

In [ ]:
## make numeric selector and verifying it works 
num_sel = make_column_selector(dtype_include='number')
num_sel(X_train)

In [ ]:
## make pipelines for categorical vs numeric data
cat_pipe = make_pipeline(SimpleImputer(strategy='constant',
                                       fill_value='MISSING'),
                         OneHotEncoder(handle_unknown='ignore', sparse=False))

num_pipe = make_pipeline(SimpleImputer(strategy='mean'))

> Nothing we have done yet should be new code. The changes we will make will be when we create our ColumnTransformer with `make_column_transformer`.
- From now on, you should add `verbose_feature_names_out=False` to `make_column_transformer`

In [ ]:
## make the preprocessing column transformer
preprocessor = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel),
                                      verbose_feature_names_out=False)
preprocessor

In [ ]:
## DELETE LATER - just demoing using classes directly
preprocessor_class = ColumnTransformer([('num',num_pipe, num_sel),
                                       ('cat',cat_pipe,cat_sel)])
preprocessor_class

>- In order to extract the feature names from the preprocessor, we first have to fit it on the data.
- Next, we can use the `preprocessor.get_feature_names_out()` method and save the output as something like "feature_names" or "final_features".

In [ ]:
## fit column transformer and run get_feature_names_out
preprocessor.fit(X_train)
feature_names = preprocessor.get_feature_names_out()
feature_names

- Notice how we were able to get the complete list of feature names, including the One Hot Encoded features with their proper "zipcode" prefix. 
- Quick note: if you forgot to add `verbose_feature_names_out` when you made your preprocessor, you would get something like this:


In [ ]:
## make the preprocessing column transformer
preprocessor_oops = make_column_transformer((num_pipe, num_sel),
                                       (cat_pipe,cat_sel)
                                           ) # forgot verbose_feature_names_out=False
## fit column transformer and run get_feature_names_out
preprocessor_oops.fit(X_train)
feature_names_oops = preprocessor_oops.get_feature_names_out()
feature_names_oops

### Remaking Our X_train and X_test as DataFrames

- Now that we have our list of feature names, we can very easily transform out X_train and X_test into preprocessed dataframes. 
- We can immediately turn the output of our preprocessor into a dataframe and do not need to save it as a separate variable first.
    - Therefore, in our pd.DataFrame, we will provide the `preprocessor.transform(X_train)` as the first argument, followed by `columns=feature_names` (the list we extracted from our precprocessor)
    - Pro Tip: you can also use the same index as your X_train or X_test variable, if you want to match up one of the transformed rows with the original dataframe.

In [ ]:
X_train_df = pd.DataFrame(preprocessor.transform(X_train), 
                          columns = feature_names, index = X_train.index)
X_train_df.head(3)

In [ ]:
X_test_df = pd.DataFrame(preprocessor.transform(X_test), 
                          columns = feature_names, index = X_test.index)
X_test_df.head(3)

In [ ]:
## confirm the first 3 rows index in y_test matches X_test_df
y_test.head(3)

- Notice that we cannot see all of our features after OneHotEncoding. Pandas truncates the display in the middle and displays `...` instead. 
- We can get around this by changing the settings in Pandas using `pd.set_option`
    - In this case, we want to change the `max_columns` to be a number larger than our number of final features. Since we have 87 features, setting the `max_columns` to 100 would be sufficient.
- For more information on pandas options, see their [documentation on Options and Settings](https://pandas.pydata.org/docs/user_guide/options.html)
- Final note: in your project notebooks, you should add this function to the top of your notebook right after your imports.

In [ ]:
## Using pd.set_option to display more columns
pd.set_option('display.max_columns',100)
X_train_df.head(3)

## Extracting Coefficients and Intercept from Scikit-Learn Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

## fitting a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_df, y_train)
print(f'Training R^2: {lin_reg.score(X_train_df, y_train):.3f}')
print(f'Test R^2: {lin_reg.score(X_test_df, y_test):.3f}')

- For scikit-learn Linear Regressions, we can find the coefficients for the features that were included in our X-data under the `.coef_` attribute. 
-  the `.coef_` is a numpy matrix that should have the same number of values as the # of columns in X_train_df

In [ ]:
## Checking the number of coeffs matches the # of feature names
print(len(lin_reg.coef_))
len(feature_names)

> Note: if for some reason the length of your coef_ is 1, you should add the `.flatten()` method to convert the  coef_ into a simple 1-D array.

### Saving the coefficients as a pandas Series

- We can immediately turn the the models' .coef_ into a pd.Series, as well.
    - Therefore, in our pd.Series, we will provide the `lin_reg.coef_` as the first argument, followed by `index=feature_names` (pandas Series are 1D and do not have columns)

In [ ]:
## Saving the coefficients
coeffs = pd.Series(lin_reg.coef_, index= feature_names)
coeffs

- The constant/intercept is not included in the .ceof_ attribute (if we used the default settings for LinearRegression which sets fit_intercept = True)
- The intercept is stored in the `.intercept_` attribute 
- We can add this as a new value to our coeffs series.
- Note: it is up to you what you name your intercept/constant. If you wanted to keep the naming convention of statsmodels, you could use "const" or just "intercept" for simplicity.

In [ ]:
# use .loc to add the intercept to the series
coeffs.loc['intercept'] = lin_reg.intercept_
coeffs

### Displaying the Coefficients

- Just like we increased the number of columns displayed by pandas, we can also increase the number of rows displayed by pandas.
- CAUTION: DO NOT SET THE MAX ROWS TO 0!! If you try to display a dataframe that has 1,000,000 it will try to display ALL 1,000,000 rows and will crash your kernel.

In [ ]:
pd.set_option('display.max_rows',100)
coeffs

### Suppressing Scientific Notation in Pandas

> We can ALSO use panda's options to change how it display numeric values.
- if we want to add a `,` separator for thousands and round to 2 decimal places, we would use the format code ",.2f". 
- In order for Pandas to use this, we will have to use an f-string with a lambda x. (X represent any numeric value being displayed by pandas).

In [ ]:
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")
coeffs

## Summary

### Recap - Sklearn v1.1

- We added the argument `verbose_feature_names_out=False` to `make_column_transformer`, which let us extract our feature names (after fitting the preprocessor) using `.get_feature_names_out()`

- We then used this list of features when reconstruction our transformed X_train and X_test as dataframes and when extracting coefficients from our model.

### Recap - Pandas Options

- We used the following options in our notebook. Ideally, we should group these together and move them to the top of our notebook, immediately after our imports.

In [ ]:
## Reviewing the options used
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")

# 📌 BOOKMARK: End Lesson?
- I can see either ending the lesson here and picking up with these coefficients in our next lesson, where we focus on visualizing the coefficients, examining some features more closely (e.g. bedrooms) and following up with scaling vs not-scaling coefficients.
- OR including the visualization in this lesson and pickup with Scaling vs Not-Scaling and Intercept vs no-intercept.

- ALSO: I am using the following code for myself, but it may be good to include here. It was at the end of week 16 and many students probably didn't see it.

## Saving Our Model for Later

- We will continue to explore the results from this model in the next lesson. 
    - We will create a dictionary of variables we want to export to use in a subsequent notebook/analysis. It will include
    
- While pickle is a common tool used for this, the joblib package has become increasing popular. Scikit-learn now promotes joblib files as the best way to save models. 

    - Here is the [section of the User Guide on "Serializing models"](https://scikit-learn.org/stable/modules/model_persistence.html#python-specific-serialization) where they demonstrate using joblib.
    
    
- To make it easy to remember which variable was which, we will save the data and model into a dictionary first.
    - We will save our:
        - Outlier removed training and test data
        - Our preprocessing column transformer
        - The scaler we used to transform price when looking for outliers 
        - Our OLS results that contain the .summary()
    - Then we will save the dictionary to a joblib file.
    

In [ ]:
## saving variables for next lesson/notebook
import joblib

export = {'X_train':X_train_df,
         'y_train': y_train,
         'X_test':X_test_df, 
         "y_test": y_test,
         'preprocessor':preprocessor,
         'model':lin_reg,
         'coeffs':coeffs}

joblib.dump(export, 'lesson01_vars.joblib')

# APPENDIX

In [ ]:
## Adding folder above to path
import os, sys
sys.path.append(os.path.abspath('../../'))

## Load stack_functions with autoreload turned on
%load_ext autoreload
%autoreload 2
from CODE import stack_functions as sf

def show_code(function):
    import inspect 
    from IPython.display import display,Markdown, display_markdown
    code = inspect.getsource(function)
    md_txt = f"```python\n{code}\n```"
    return display(Markdown(md_txt))

# 📌 BOOKMARK 2: Outliers

## Outlier/Anomaly Detection & Removal (with sklearn)

- User Guide:
    - https://scikit-learn.org/stable/modules/outlier_detection.html
- Models:
    - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html#sklearn.ensemble.IsolationForest
    - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html#sklearn.neighbors.LocalOutlierFactor

### Model WITH Outliers Included

In [ ]:
from sklearn.linear_model import LinearRegression

## ORIGINAL LINREG
## fitting a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_df, y_train)
print(f'Training R^2: {lin_reg.score(X_train_df, y_train):.3f}')
print(f'Test R^2: {lin_reg.score(X_test_df, y_test):.3f}')
sf.evaluate_ols(lin_reg, X_train_df, y_train)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

### Isolation Forest

In [ ]:
detector = IsolationForest()
detector.fit(X_train_df, y_train)

output_train = detector.predict(X_train_df)
# output_test = detector.predict(X_test_df)

display(pd.Series(output_train, name='Train Outliers').value_counts())#,
# pd.Series(output_test, name='Test Outliers').value_counts())

> Trying isolation forest with y vars added

In [ ]:
df_train = pd.concat([X_train_df, y_train],axis=1)
df_test = pd.concat([X_test_df, y_test],axis=1)
display(df_train.head(3), df_test.head(3))

In [ ]:
detector = IsolationForest()
detector.fit(df_train)

output_train = detector.predict(df_train)
# output_test = detector.predict(df_test)

display(pd.Series(output_train, name='Train Outliers').value_counts())#,
# pd.Series(output_test, name='Test Outliers').value_counts())

### Local Outlier Factor [UserGuide](https://scikit-learn.org/stable/modules/outlier_detection.html#local-outlier-factor)

In [ ]:
detector = LocalOutlierFactor()
detector.fit_predict(X_train_df)#, y_train)

output_train = pd.Series(detector.fit_predict(X_train_df), 
                         index=X_train_df.index,
                        name='Train Outliers')
# output_test = pd.Series(detector.predict(X_test_df), index=X_test_df.index,
#                        name='Test Outliers')

display(output_train.value_counts())#, output_test.value_counts())

In [ ]:
## Viewing outliers in training data
X_train_df[ output_train ==-1]

In [ ]:
## Keeping only good rows
X_train_cln = X_train_df.loc[output_train==1]
y_train_cln = y_train.loc[ output_train==1]
X_train_cln

In [ ]:
# X_test_cln = X_test_df.loc[output_test==1]
# y_test_cln = y_test.loc[output_test==1]
# X_test_cln

#### Model wtih LocalOutlierFactor outliers removed

In [ ]:
## fitting a linear regression model
lin_reg_cln = LinearRegression()
lin_reg_cln.fit(X_train_cln, y_train_cln)
print(f'Training R^2: {lin_reg_cln.score(X_train_cln, y_train_cln):.3f}')
print(f'Test R^2: {lin_reg_cln.score(X_test_df, y_test):.3f}')
sf.evaluate_ols(lin_reg_cln, X_train_cln, y_train_cln)
sf.evaluate_ols(lin_reg_cln, X_test_df, y_test)

### Removing Outliers, including target

In [ ]:
df_train = pd.concat([X_train_df, y_train],axis=1)
# df_test = pd.concat([X_test_df, y_test],axis=1)
display(df_train.head(3))#, df_test.head(3))

In [ ]:
## trying concat vars
detector = LocalOutlierFactor()#novelty=True)
output_train = pd.Series(detector.fit_predict(X_train_df), 
                         index=X_train_df.index,
                        name='Train Outliers')

display(output_train.value_counts())

In [ ]:
## Keeping only good rows
X_train_cln = X_train_df.loc[output_train==1]
y_train_cln = y_train.loc[ output_train==1]
# X_test_cln = X_test_df.loc[output_test==1]
# y_test_cln = y_test.loc[output_test==1]
# X_test_cln
X_train_cln

In [ ]:
## fitting a linear regression model
lin_reg_cln = LinearRegression()
lin_reg_cln.fit(X_train_cln, y_train_cln)
print(f'Training R^2: {lin_reg_cln.score(X_train_cln, y_train_cln):.3f}')
print(f'Test R^2: {lin_reg_cln.score(X_test_df, y_test):.3f}')
sf.evaluate_ols(lin_reg_cln, X_train_cln, y_train_cln)
sf.evaluate_ols(lin_reg_cln, X_test_df, y_test)

In [ ]:
## ORIGINAL LINREG
## fitting a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_df, y_train)
print(f'Training R^2: {lin_reg.score(X_train_df, y_train):.3f}')
print(f'Test R^2: {lin_reg.score(X_test_df, y_test):.3f}')
sf.evaluate_ols(lin_reg, X_train_df, y_train)
sf.evaluate_ols(lin_reg, X_test_df, y_test)

### Comparing with IQR Rule

In [ ]:
show_code(sf.remove_outliers)

In [ ]:
df_train = pd.concat([X_train_df, y_train],axis=1)
df_train

In [ ]:
outlier_cols = [c for c in df_train.columns if not c.startswith('zipcode')]
outlier_cols

In [ ]:
df_train_cln = sf.remove_outliers(df_train, subset=outlier_cols)

X_train_iqr = df_train_cln.drop(columns='price')
y_train_iqr = df_train_cln['price']
X_train_iqr

In [ ]:
## fitting a linear regression model
lin_reg_cln = LinearRegression()
lin_reg_cln.fit(X_train_iqr, y_train_iqr)
print(f'Training R^2: {lin_reg_cln.score(X_train_iqr, y_train_iqr):.3f}')
print(f'Test R^2: {lin_reg_cln.score(X_test_df, y_test):.3f}')
sf.evaluate_ols(lin_reg_cln, X_train_iqr, y_train_iqr)
sf.evaluate_ols(lin_reg_cln, X_test_df, y_test)

In [ ]:
# ## fitting a linear regression model
# lin_reg_cln = LinearRegression()
# lin_reg_cln.fit(X_train_cln, y_train_cln)
# print(f'Training R^2: {lin_reg_cln.score(X_train_cln, y_train_cln):.3f}')
# print(f'Test R^2: {lin_reg_cln.score(X_test_df, y_test):.3f}')
# sf.evaluate_ols(lin_reg_cln, X_train_cln, y_train_cln)
# sf.evaluate_ols(lin_reg_cln, X_test_df, y_test)

## Testing if Zipcode Good for Rare Label Encoding

#### DELETE ME - Testing If zipcode would work for demo'ing rare label encoding


In [ ]:
## DELETE ME - Testing If zipcode would work for demo'ing rare label encoding
zip_counts = df['zipcode'].value_counts(1)
zip_counts#.cumsum().plot()

In [ ]:
## DELETE ME
zip_counts[zip_counts<.05]

In [ ]:
## DELETE ME
sns.histplot(zip_counts)

In [ ]:
## DELETE ME - Testing If zipcode would work for demo'ing rare label encoding
zip_counts.plot(kind='barh',figsize=(8,14))